In [1]:
def gen_new_headers(old_headers):
    headers = ['colors_' + str(x+1) + '_' for x in range(10)]
    h = []
    for x in headers:
        h.append(x + 'red')
        h.append(x + 'blue')
        h.append(x + 'green')
    return old_headers + h + ['genre']

In [ ]:
from analyzer import get_mode_colors
import pandas as pd
from os import listdir,fsync
import csv
import time

# this function is needed because of mismatch between file system videos and
# md csv information
def get_file_names_from_md_csv(genre):
    meta_csv_file = open('../Videos2/metadata/' + genre + '2_md.csv', 'r')
    meta_csv = csv.reader(meta_csv_file)
    meta_csv.next()  # header line
    file_names = []
    for row in meta_csv:
        file_names.append(row[1])
    return file_names

time.sleep(30*60)
genres = ['country', 'pop', 'rap', 'rock']
for genre in genres:
    directory = '../Videos2/compressed_' + genre    
    meta_csv_file = open('../Videos2/metadata/' + genre + '2_md.csv', 'r')
    meta_csv = csv.reader(meta_csv_file)
    new_csv_file = open('../Videos2/all_data/' + genre + '_all.csv', 'w')
    new_csv = csv.writer(new_csv_file)
    
    old_headers = meta_csv.next()
    new_headers = gen_new_headers(old_headers)
    
    new_csv.writerow(new_headers)
    
    vid_files = listdir(directory)
    md_files = get_file_names_from_md_csv(genre)

    for file_name in md_files:
        print file_name
        if file_name in vid_files:  # if false, the video does not exist
            try:
                color = get_mode_colors(directory + '/' + file_name)
                color = map(str, map(int, color))
                new_row = meta_csv.next() + color + [genre]
                print ''
                print str(file_name)
                print new_row
                # write the file as we go to avoid losing data
                new_csv.writerow(new_row)
                new_csv_file.flush()
                fsync(new_csv_file.fileno())
                time.sleep(1)
                x += 1
            except:
                print "probably caught a black and white video"
        else:
            print file_name
    new_csv_file.close()
    meta_csv_file.close()
    time.sleep(60*30)